In [ ]:
#mount gdrive
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow import keras
import pandas as pd
import pickle 
import datetime, os
import tensorflow as tf
import re

review = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset/Output_dataset/review_w_sentiment.csv', lineterminator='\n', encoding='utf8')

Here i predefined a tag_dict to label the data, for the labels [0,1,2,3,4] represents no problem, crash problem, music playing problem, account problem and other problem correspondingly.

Note that for all reviews whose rating>2 are all labeled no problem (0).

In [ ]:
#crash,播放,帳號,其他
tag_dict={1:['閃退','崩潰','跑不動','關掉','卡頓','卡死','退','關閉'],
          2:['播','聲音','播放','排序','聲音','音量','音軌','暫停','斷掉','一半','放歌','斷開','撥'],
          3:['登錄','登入','轉圈','斷線','登','登出','黑頻','註冊','帳號','密碼','連線']}

Here i check if the review whose rating<=2 exisit a keyword within, if so, the class counter add 1, finally, if the class counter==1 or ==0, it means the review has only one specific class, if the class counter>1, it means the review might contains information of multiple class, for these cases, I labeled them -1 and artificially labeld them outside the program. (*It's subjective but help prevent from tagging the wrong label)

In [ ]:
review['_class']=0
multi_class_index=[]
for i in range(len(review)):
  if review['rating'][i]>2:
    continue
  class_ctr=0
  current_class=0
  for k in range(len(review['segment_content'][i])):
    if review['segment_content'][i][k] in tag_dict[1]:
      class_ctr+=1
      current_class=1
    if review['segment_content'][i][k] in tag_dict[2]:
      class_ctr+=1
      current_class=2
    if review['segment_content'][i][k] in tag_dict[3]:
      class_ctr+=1
      current_class=3
  if class_ctr==0:
    review.at[i,'_class']=4
  if class_ctr==1:
    review.at[i,'_class']=current_class
  if class_ctr>1:
    review.at[i,'_class']=-1
    multi_class_index.append(i)
    

In [ ]:
#mutilable, requires labor to check/define the class
review.loc[review._class==-1]

,content,rating,sentiment,original_content,segment_content,_class
39,我 的 手機 型號 是 最近 更新 後 只要 每 次 開啟 聽 歌 時 都 會 閃退 而且 ...,2,-0.575000,我的手機型號是最近更新後只要每次開啟聽歌時都會閃退而且已經好幾次了而且每次重開都會閃退已經連...,"[['我', '的', '手機', '型號', '是', '最近', '更新', '後', ...",-1
73,近期 非常 頻繁 出現 不 正常 關閉 的 視窗 但 視窗 跳出 時 音樂 還 在 正常 播...,2,-0.160000,近期非常頻繁出現不正常關閉的視窗但視窗跳出時音樂還在正常播放播直到我按掉視窗才會關掉另一個問...,"[['近期', '非常', '頻繁', '出現', '不', '正常', '關閉', '的'...",-1
133,最近 播放 過程 容易 出現 無 預警 閃退 請 協助 改善,1,-0.550000,最近播放過程容易出現無預警閃退請協助改善,"[['最近', '播放', '過程', '容易', '出現', '無', '預警', '閃退...",-1
146,今天 下午 要 開 的 時候 一直 閃退 然後 重新 下載 後 登入 帳號 都 沒 錯 可是...,1,-0.566667,今天下午要開的時候一直閃退然後重新下載後登入帳號都沒錯可是我的歌單全部不見一千兩百多首歌都不見了,"[['今天', '下午', '要', '開', '的', '時候', '一直', '閃退',...",-1
502,莫名其妙 無法 上線 登出 後 還 不能 登入,1,-0.400000,莫名其妙無法上線登出後還不能登入,"[['莫名其妙', '無法', '上線', '登出', '後', '還', '不能', '登...",-1
550,為何 無法 登入 按 註冊 顯示 已 是 會員 按 登入 後 一直 跳到 註冊 頁面,2,-0.400000,為何無法登入按註冊顯示已是會員按登入後一直跳到註冊頁面,"[['為何', '無法', '登入', '按', '註冊', '顯示', '已', '是',...",-1
583,我 不 明白 為何 我 的 全部 歌曲 首離線 歌曲 有首 可以 聽 這 是 什麼 概念 希...,2,-0.414286,我不明白為何我的全部歌曲首離線歌曲有首可以聽這是什麼概念希望新增功能未授權直接自動刪除不用在...,"[['我', '不', '明白', '為何', '我', '的', '全部', '歌曲', ...",-1
594,爛死 了 我 都 註冊 過 了 還 不 讓 我 登入 一直 要 我 註冊 下面 在 跟 我 ...,1,-0.700000,爛死了我都註冊過了還不讓我登入一直要我註冊下面在跟我說我註冊過了又不讓我登入是有病嗎,"[['爛死', '了', '我', '都', '註冊', '過', '了', '還', '不...",-1
637,登錄 很 久 常常 無法 登錄 快 不 想 用 了,2,-0.500000,登錄很久常常無法登錄快不想用了,"[['登錄', '很', '久', '常常', '無法', '登錄', '快', '不', ...",-1
732,我 不 想 升級 白金 會員 但是 音樂 卻 無法 完整 的 整 首 播放 只 能 播 秒 ...,2,-0.200000,我不想升級白金會員但是音樂卻無法完整的整首播放只能播秒請問這個問題是正常的嗎還是有什麼可以改...,"[['我', '不', '想', '升級', '白金', '會員', '但是', '音樂',...",-1


Note that this saved .csv isn't the final version, I tag them afterwards.

In [ ]:
review.to_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset/Output_dataset/review_w_class.csv',encoding='utf-8',index=False)